In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

### load sample of polypharmacy side effect dataset
Every element of sample is node pair

In [ ]:
sample = []

file_sample = open(".\\polypharmacy side effect dataset\\sample.txt").readlines()

for line in file_sample:
    
    sample.append((int(line.split(" ")[0]), int(line.split(" ")[1])))
    
del file_sample

### load conduit type group for sample
1 denotes the samples in rank from 1 to 30k;

2 denotes the samples in rank from 30k to 50k;

3 denotes the samples in rank from 50k to 63472.

In [ ]:
group = []

file_group = open(".\\polypharmacy side effect dataset\\group.txt").readlines()

for line in file_group:
    
    group.append(int(line))
    
del file_group

### load adjacent matrix and build norm adjacent matrix of drug

In [ ]:
adj_matrix = np.load(".\\polypharmacy side effect dataset\\adj_matrix.npy")

In [ ]:
node_num = adj_matrix.shape[0]

degree_matrix = np.zeros((node_num, node_num))

for i in range(node_num):
        
    degree_matrix[i,i] = np.sum(adj_matrix[i])

In [ ]:
norm_degree_matrix = np.zeros((node_num, node_num))

for i in range(node_num):

    norm_degree_matrix[i, i] = degree_matrix[i, i]**(-1 / 2)

middel_adj_matrix = np.dot(norm_degree_matrix, adj_matrix)

norm_adj_matrix = np.dot(middel_adj_matrix, norm_degree_matrix)

del middel_adj_matrix

In [ ]:
norm_adj_matrix = torch.from_numpy(norm_adj_matrix)

### Save the degree of drug node

In [ ]:
degree = []

for i in range(node_num):
    
    degree.append(degree_matrix[i, i])
    
del degree_matrix

del adj_matrix

### Stage1: node learning
in_size is the dimension of the features of the previous layer ($l-1$).

out_size is the dimension of the feature of the current layer ($l$).

old_node_embedding is the feature of the previous layer ($l-1$).

adj_mat is the norm adjacent matrix.

use_divce denotes whether CUDA is used.

In [ ]:
class node_learning(nn.Module):
    def __init__(self,
                 in_size,
                 out_size):

        super(node_learning, self).__init__()

        self.nei_update1 = nn.Linear(in_size, out_size, bias=False)

        self.self_node_update1 = nn.Linear(in_size, out_size, bias=False)
         
    def forward(self,
                out_size,
                adj_mat,
                old_node_embedding,
                use_divce):
        
        total_node = old_node_embedding.shape[0]
    
        if use_divce == 1:
            
            new_node_embedding = torch.zeros((total_node, out_size)).cuda()
            
        else:
            
            new_node_embedding = torch.zeros((total_node, out_size)).cpu()
            
        for node in range(total_node):
            
            nei_information, nei_gather = 0, 0
            
            self_information, total_information = 0, 0
                    
            self_information += self.self_node_update1(
                        old_node_embedding[node, :])
                    
            nei_gather += torch.mm(adj_mat[node, :].reshape(1, -1),
                                           old_node_embedding)
                    
            nei_information += self.nei_update1(nei_gather)
                    
            total_information += F.relu(nei_information +
                                                       self_information)
                
            new_node_embedding[node, :] = torch.add(
                new_node_embedding[node, :], total_information)
            
        return new_node_embedding

### Stage2: conduit node learning
pre_size is the dimension of the features of the previous layer ($l-1$).

next_size is the dimension of the feature of the current layer ($l$).

node_embedding is the feature matrix of drug node.

use_divce denotes whether CUDA is used.

In [ ]:
class conduit_node_learning(nn.Module):
    
    def __init__(self,
                 pre_size,
                 next_size):

        super(conduit_node_learning, self).__init__()

        self.left_gate1 = nn.Linear(pre_size, next_size, bias=False)

        self.right_gate1 = nn.Linear(pre_size, next_size, bias=False)

        self.left_gate2 = nn.Linear(pre_size, next_size, bias=False)
        
        self.right_gate2 = nn.Linear(pre_size, next_size, bias=False)
        
        self.left_gate3 = nn.Linear(pre_size, next_size, bias=False)
        
        self.right_gate3 = nn.Linear(pre_size, next_size, bias=False)
        
        self.c1_conduit_update = nn.Linear(pre_size,
                                                  next_size,
                                                  bias=True)
        
        self.c2_conduit_update = nn.Linear(pre_size,
                                                   next_size,
                                                   bias=True)
        
        self.c3_conduit_update = nn.Linear(pre_size,
                                            next_size,
                                            bias=True)
    
    def forward(self, 
                sample,
                group,
                next_size, 
                node_embedding, 
                use_divce):
        
        conduit_sample = sample
        
        index = list(range(len(conduit_sample)))
        
        if use_divce == 1:
            
            new_conduit_embedding = torch.zeros(
                (len(conduit_sample), next_size)).cuda()
            
        else:
            
            new_conduit_embedding = torch.zeros(
                (len(conduit_sample), next_size)).cpu()
            
        for j in index:
            
            i = conduit_sample[j]
            
            left_gate, right_gate, conduit_embedding = 0, 0, 0
            
            middle_state, gather_information = 0, 0
            
            gather_information += torch.add(node_embedding[i[0], :],
                                            node_embedding[i[1], :])
            
            if group[j] == 1:
                
                left_gate += torch.sigmoid(
                    self.left_gate1(node_embedding[i[0], :]))
                
                right_gate += torch.sigmoid(
                    self.right_gate1(node_embedding[i[1], :]))
                
                middle_state += torch.tanh(
                    self.c1_conduit_update(gather_information))
                
            if group[j] == 2:
                
                left_gate += torch.sigmoid(
                    self.left_gate2(node_embedding[i[0], :]))
                
                right_gate += torch.sigmoid(
                    self.right_gate2(node_embedding[i[1], :]))
                
                middle_state += torch.tanh(
                    self.c2_conduit_update(gather_information))
                
            if group[j] == 3:
                
                left_gate += torch.sigmoid(
                    self.left_gate3(node_embedding[i[0], :]))
                
                right_gate += torch.sigmoid(
                    self.right_gate3(node_embedding[i[1], :]))
                
                middle_state += torch.tanh(
                    self.c3_conduit_update(gather_information))
            
            conduit_embedding += left_gate * middle_state + right_gate * middle_state
            
            new_conduit_embedding[j, :] += conduit_embedding.reshape(-1)
       
        return new_conduit_embedding

### The framework of CGNN
The dimensions of each layer are the same as those in paper.

pre_node_embedding is the initial feature of drug node.

adj_mat is the norm adjacent matrix.

self.gather_1 is the weight matrix of layer-wise updating rule.

use_divce denotes whether CUDA is used.

In [ ]:
class conduitGNN(nn.Module):
    
    def __init__(self):
        
        super(conduitGNN, self).__init__()
        
        self.node_update_layer1 = node_learning(256, 128)
        
        self.conduit_layer1 = conduit_node_learning(128, 964)
        
        self.node_update_layer2 = node_learning(128, 64)
        
        self.conduit_layer2 = conduit_node_learning(64, 964)
        
        self.gather_1 = nn.Linear(964, 964, bias=True)
        
    def forward(self,
                pre_node_embedding,
                adj_mat,
                sample,
                group,
                use_divce):
        
        use_divce = use_divce
        
        if use_divce == 1:
            
            output = torch.zeros((len(sample), 964)).cuda()
            
            pre_node_embedding = pre_node_embedding.cuda()
            
        else:
            
            output = torch.zeros((len(sample), 964)).cpu()
            
            pre_node_embedding = pre_node_embedding.cpu()
            
        ##########

        node_embedding_1 = self.node_update_layer1(128,
                                                   adj_mat,
                                                   pre_node_embedding,
                                                   use_divce)

        conduit_embedding_1 = self.conduit_layer1(sample, 
                                                  group,
                                                  964,
                                                  node_embedding_1, 
                                                  use_divce)
        
        #########
        node_embedding_2 = self.node_update_layer2(64,
                                                   adj_mat,
                                                   node_embedding_1,
                                                   use_divce)
        
        conduit_embedding_2 = self.conduit_layer2(sample,
                                                  group,
                                                  964,
                                                  node_embedding_2,
                                                  use_divce)
        #########
        
        
        gather_conduit1 = torch.sigmoid(
            self.gather_1(conduit_embedding_1) + conduit_embedding_2)
        
        #########
        
        output += gather_conduit1
        
        return output

### Computing acc for each side effect and average all acc as final result.

In [ ]:
def class_acc(p_mat, plabel_mat, nlabel_mat):
    
    pre_p = (p_mat > 0.5).double()*plabel_mat
    
    pre_n = (p_mat <= 0.5).double()*nlabel_mat
    
    acc_list = []
    
    for c in range(p_mat.shape[1]):
        
        tp_num = torch.sum(pre_p[:, c])
        
        edge1_num = torch.sum(plabel_mat[:, c])
        
        tn_num = torch.sum(pre_n[:, c])
        
        edge2_num = torch.sum(nlabel_mat[:, c])
        
        acc = (tp_num + tn_num)/(edge1_num + edge2_num)
        
        acc_list.append(acc.item())
        
    return acc_list

### Compute loss for each side effect and sum all loss as total loss

In [ ]:
def my_loss(p_mat, plabel_num, nlabel_num, plabel_mat, nlabel_mat, weight):

    loss, loss_list = 0, []

    for i in range(p_mat.shape[1]):

        pos_p = np.array(torch.nonzero(plabel_mat[:, i]))

        pos_p = np.random.choice(pos_p.reshape(-1),
                               int(0.5 * plabel_num[i]),
                               replace=False)

        plabel = torch.ones(1, int(0.5 * plabel_num[i]))
        
        pos_n = np.array(torch.nonzero(nlabel_mat[:, i]))

        pos_n = np.random.choice(pos_n.reshape(-1),
                               int(0.5 * nlabel_num[i]),
                               replace=False)
        
        nlabel = torch.zeros((1, int(0.5 * nlabel_num[i])))
        
        pos = np.hstack((pos_p, pos_n))
        
        pos = torch.from_numpy(pos)
        
        label = torch.cat((plabel, nlabel), dim=1).reshape(-1)
        
        loss += F.binary_cross_entropy(p_mat[pos, i], label) * weight[i]
        
        loss_list.append((F.binary_cross_entropy(p_mat[pos, i], label) * weight[i]).item())

    return loss, loss_list

In [ ]:
def regu():
    
    reg_loss = 0
    
    for name,param in conduit_GNN.named_parameters():
        
        if 'conduit_update' in name and 'weight' in name:
            
            l2_reg = torch.norm(param,p=2)
            
            reg_loss += 0.005*l2_reg
            
        if 'node_update' in name and 'weight' in name:
            
            l2_reg = torch.norm(param,p=2)
            
            reg_loss += 0.005*l2_reg
            
    return reg_loss

### load feature of drug node and initialize CGNN model

In [ ]:
all_node_embedding = torch.load('/data/Decagon/dataset/all_node_embedding.pth')

conduit_GNN = conduitGNN()

optm = torch.optim.Adam(conduit_GNN.parameters(), lr=0.005)

for name,param in conduit_GNN.named_parameters():
    
    print(name)

### load positive label and negative label for train sample, validation sample and test sample

In [ ]:
train_posi_label = np.load(".\\polypharmacy side effect dataset\\train posi label.npy")

val_posi_label = np.load(".\\polypharmacy side effect dataset\\val posi label.npy")

test_posi_label = np.load(".\\polypharmacy side effect dataset\\test posi label.npy")

train_nega_label = np.load(".\\polypharmacy side effect dataset\\train nega label.npy")

val_nega_label = np.load(".\\polypharmacy side effect dataset\\val nega label.npy")

test_nega_label = np.load(".\\polypharmacy side effect dataset\\test nega label.npy")

In [ ]:
train_posi_label = torch.from_numpy(train_posi_label)

val_posi_label = torch.from_numpy(val_posi_label)

test_posi_label = torch.from_numpy(test_posi_label)

train_nega_label = torch.from_numpy(train_nega_label)

val_nega_label = torch.from_numpy(val_nega_label)

test_nega_label = torch.from_numpy(test_nega_label)

In [ ]:
posi_label_num = []

for c in range(train_posi_label.shape[1]):
    
    posi_label_num.append(torch.sum(train_posi_label[:, c]).item())
    
nega_label_num = []

for c in range(train_nega_label.shape[1]):
    
    nega_label_num.append(torch.sum(train_nega_label[:, c]).item())

### Run the following code to get the train acc, validation acc and test acc.

In [ ]:
valid_acc_list,test_acc_list = [],[]

# exper_num = ''

total_start_time = time.time()

for epoch in range(30):
    
    loss,train_acc = 0,0
        
    conduit_GNN.train()
    
    conduit_GNN.cuda()
    
    start_loss_time = time.time()

    train_output_matrix = conduit_GNN(all_node_embedding,
                                      norm_adj_matrix.float().cuda(),
                                      sample,
                                      group,
                                      use_divce = 1)
        
    train_acc_for_class = class_acc(train_output_matrix.cpu(),
                              train_posi_label,
                              train_nega_label)
    
#     file_acc = open('.\\'+exper_num+'\\'+str(epoch)+'train acc for class.txt','w')

#     for i in train_acc_for_class:
    
#         file_acc.write(str(i.item())+'\n')
    
#     file_acc.close()
        
    weight = 964 * [1]
        
    loss, loss_list = my_loss(train_output_matrix.cpu(),
                        posi_label_num,
                        nega_label_num,
                        train_posi_label, 
                        train_nega_label,
                        weight)
        
    loss = loss.cuda() + regu().cuda()
    
    end_loss_time = time.time()
    
    train_acc = np.mean(train_acc_for_class)
    
    print('Epoch :',epoch,'loss time',int(end_loss_time-start_loss_time),'loss:',loss.item(),'  train acc:',train_acc.item())
    
    start_optm_time = time.time()
    
    optm.zero_grad()
    
    loss.backward()
    
    optm.step()
    
    end_optm_time = time.time()
    
    print('Epoch :',epoch,'optm time',int(end_optm_time-start_optm_time))
    
    if epoch > 5:#
        
        conduit_GNN.cpu()
        
        conduit_GNN.eval()
      
        valid_acc_for_class = class_acc(train_output_matrix.cpu(),
                                                 val_posi_label,
                                                 val_nega_label)
        
        validation_acc = np.mean(valid_acc_for_class)

        if valid_acc_list != []:
            
            val_maxacc = max(valid_acc_list)
            
        else:
            
            val_maxacc = 0
        
        valid_acc_list.append(validation_acc)
        
        print('Epoch :',epoch,'validation acc',validation_acc.item())
        
        if validation_acc > val_maxacc:
            
#             torch.save(train_output_matrix.cpu(),
#                         '.\\'+exper_num+'\\'+str(epoch)+'train_output_matrix.pth')
        
            test_acc_for_class = class_acc(train_output_matrix.cpu(), test_posi_label, test_nega_label)
        
            test_acc = np.mean(test_acc_for_class)
        
            test_acc_list.append(test_acc)
        
            print('Epoch :',epoch,'test acc',test_acc.item())
            
    del train_output_matrix
    
    torch.cuda.empty_cache()
        
total_end_time = time.time()

print('total time',int(total_end_time-total_start_time))